##### データセットの場所やバッチサイズなどの定数値の設定

In [ ]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'


# 使用するデバイス
# GPU を使用しない環境（CPU環境）で実行する場合は DEVICE = 'cpu' とする
DEVICE = 'cuda:0'

# 全ての訓練データを一回ずつ使用することを「1エポック」として，何エポック分学習するか
N_EPOCHS = 20

# 学習時のバッチサイズ
BATCH_SIZE = 100

# 訓練データセット（画像ファイルリスト）のファイル名
DATASET_CSV = './tinyCelebA/train_list.csv'

# 画像ファイルの先頭に付加する文字列（データセットが存在するディレクトリのパス）
DATA_DIR = './tinyCelebA'

# 取り扱う属性ラベル
TARGET_ATTRIBUTES = ['Blond_Hair', 'Brown_Hair', 'Black_Hair', 'Gray_Hair', 'Eyeglasses', 'Male', 'Young']

# 画像サイズ
H = 128 # 縦幅
W = 128 # 横幅
C = 3 # チャンネル数（カラー画像なら3，グレースケール画像なら1）

# 特徴ベクトルの次元数
N = 256

# 学習結果の保存先フォルダ
MODEL_DIR = './CVAE_models'

# 学習結果のニューラルネットワークの保存先
MODEL_FILE_ENC = os.path.join(MODEL_DIR, 'cvae_encoder_model.pth') # エンコーダ
MODEL_FILE_DEC = os.path.join(MODEL_DIR, 'cvae_decoder_model.pth') # デコーダ

##### ニューラルネットワークモデルの定義

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# Residual Block
class ResBlock(nn.Module):
    def __init__(self, channels, kernel_size, stride, padding, activation=F.relu):
        super(ResBlock, self).__init__()
        self.activation = activation
        self.conv1 = nn.Conv2d(in_channels=channels, out_channels=channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.conv2 = nn.Conv2d(in_channels=channels, out_channels=channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.bn1 = nn.BatchNorm2d(num_features=channels)
        self.bn2 = nn.BatchNorm2d(num_features=channels)
    def forward(self, x):
        h = self.activation(self.bn1(self.conv1(x)))
        h = self.bn2(self.conv2(h))
        return self.activation(h + x)


# 顔画像を N 次元の特徴ベクトルへと圧縮するニューラルネットワーク（CVAE版）
# CVAEのエンコーダ部分のサンプル
class FaceEncoder(nn.Module):

    # C: 入力顔画像のチャンネル数（1または3と仮定）
    # H: 入力顔画像の縦幅（16の倍数と仮定）
    # W: 入力顔画像の横幅（16の倍数と仮定）
    # N: 出力の特徴ベクトルの次元数
    # K: 属性ラベルの種類数
    def __init__(self, C, H, W, N, K):
        super(FaceEncoder, self).__init__()

        # 畳込み層1
        # カーネルサイズ3，ストライド幅1，パディング1の設定なので，これを通しても特徴マップの縦幅・横幅は変化しない
        self.conv1 = nn.Conv2d(in_channels=C, out_channels=16, kernel_size=3, stride=1, padding=1)

        # 畳込み層2～5
        # カーネルサイズ4，ストライド幅2，パディング1の設定なので，これらを通すことにより特徴マップの縦幅・横幅がそれぞれ 1/2 になる
        # 4つ適用することになるので，最終的には都合 1/16 になる -> ゆえに，入力顔画像の縦幅と横幅を各々16の倍数と仮定している
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2, padding=1)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4, stride=2, padding=1)
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=4, stride=2, padding=1)

        # バッチ正規化層
        self.bn1 = nn.BatchNorm2d(num_features=16)
        self.bn2 = nn.BatchNorm2d(num_features=32)
        self.bn3 = nn.BatchNorm2d(num_features=64)
        self.bn4 = nn.BatchNorm2d(num_features=128)
        self.bn5 = nn.BatchNorm2d(num_features=128)

        # 平坦化
        self.flat = nn.Flatten()

        # 画像情報を処理する全結合層
        # 畳込み層1～5を通すことにより特徴マップの縦幅・横幅は都合 1/16 になっている．
        # したがって，入力側のパーセプトロン数は 128*(H/16)*(W/16) = H*W/2
        self.fc_img = nn.Linear(in_features=H*W//2, out_features=256) # 画像情報は最終的に256次元に

        # 属性ラベル情報を処理する全結合層
        self.fc_lab1 = nn.Linear(in_features=K, out_features=256)
        self.fc_lab2 = nn.Linear(in_features=256, out_features=256) # ラベル情報も256次元に

        # 画像・属性ラベル情報の結合後に用いる全結合層
        self.fc_mu = nn.Linear(in_features=512, out_features=N) # 256次元になった画像情報と属性ラベル情報を結合するので，トータルで512次元
        self.fc_lnvar = nn.Linear(in_features=512, out_features=N)

    def forward(self, x, y):

        # 画像情報 x を処理
        h = F.leaky_relu(self.bn1(self.conv1(x)))
        h = F.leaky_relu(self.bn2(self.conv2(h)))
        h = F.leaky_relu(self.bn3(self.conv3(h)))
        h = F.leaky_relu(self.bn4(self.conv4(h)))
        h = F.leaky_relu(self.bn5(self.conv5(h)))
        h = self.flat(h)
        hx = torch.tanh(self.fc_img(h))

        # 属性ラベル情報 y を処理
        h = F.leaky_relu(self.fc_lab1(y))
        hy = torch.tanh(self.fc_lab2(h))

        # 画像情報と属性ラベル情報を結合
        h = torch.cat((hx, hy), dim=1)

        # 特徴分布の平均・分散を計算し，特徴ベクトルを一つサンプリング
        mu = self.fc_mu(h)
        lnvar = self.fc_lnvar(h)
        eps = torch.randn_like(mu) # mu と同じサイズの標準正規乱数を生成
        z = mu + eps * torch.exp(0.5 * lnvar)
        return z, mu, lnvar


# N 次元の特徴ベクトルから顔画像を生成するニューラルネットワーク（CVAE版）
# CVAEのデコーダ部分のサンプル
class FaceDecoder(nn.Module):

    # C: 出力顔画像のチャンネル数（1または3と仮定）
    # H: 出力顔画像の縦幅（16の倍数と仮定）
    # W: 出力顔画像の横幅（16の倍数と仮定）
    # N: 入力の特徴ベクトルの次元数
    # K: 属性ラベルの種類数
    def __init__(self, C, H, W, N, K):
        super(FaceDecoder, self).__init__()
        self.W = W
        self.H = H

        # 属性ラベル情報を処理する全結合層
        self.fc_lab1 = nn.Linear(in_features=K, out_features=256)
        self.fc_lab2 = nn.Linear(in_features=256, out_features=256) # 属性ラベル情報は最終的に256次元に

        # 特徴ベクトルを処理する全結合層
        self.fc_feat = nn.Linear(in_features=N, out_features=256) # 特徴ベクトルも256次元に

        # 属性ラベル情報と特徴ベクトルの統合後に用いる全結合層
        self.fc_all = nn.Linear(in_features=512, out_features=H*W//2) # 256次元になったラベル情報と特徴ベクトルを結合するので，トータルで512次元

        # 逆畳込み層1～4
        # カーネルサイズ，ストライド幅，パディングは FaceEncoder の畳込み層2～5と真逆に設定
        self.deconv1 = nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=4, stride=2, padding=1)
        self.deconv2 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=4, stride=2, padding=1)
        self.deconv3 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=4, stride=2, padding=1)
        self.deconv4 = nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=4, stride=2, padding=1)

        # Residual Block 1〜4
        # checker board artifact の補正を狙いとして逆畳み込み層の直後に入れる
        self.rb1 = ResBlock(channels=128, kernel_size=3, stride=1, padding=1)
        self.rb2 = ResBlock(channels=64, kernel_size=3, stride=1, padding=1)
        self.rb3 = ResBlock(channels=32, kernel_size=3, stride=1, padding=1)
        self.rb4 = ResBlock(channels=16, kernel_size=3, stride=1, padding=1)

        # バッチ正規化層
        self.bn1 = nn.BatchNorm2d(num_features=128)
        self.bn2 = nn.BatchNorm2d(num_features=64)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        self.bn4 = nn.BatchNorm2d(num_features=16)

        # 畳込み層（最終層）
        self.conv = nn.Conv2d(in_channels=16, out_channels=C, kernel_size=3, stride=1, padding=1)

    def forward(self, z, y):

        # 特徴ベクトル z を処理
        hz = torch.tanh(self.fc_feat(z))

        # 属性ラベル情報 y を処理
        h = F.leaky_relu(self.fc_lab1(y))
        hy = torch.tanh(self.fc_lab2(h))

        # 特徴ベクトルと属性ラベル情報を結合
        h = torch.cat((hz, hy), dim=1)

        # 画像を生成
        h = F.leaky_relu(self.fc_all(h))
        h = torch.reshape(h, (len(h), 128, self.H//16, self.W//16)) # 一列に並んだパーセプトロンを 128*(H/16)*(W/16) の特徴マップに並べ直す
        h = F.leaky_relu(self.bn1(self.deconv1(h)))
        h = self.rb1(h) # Residual Blockの内部でバッチ正規化と活性化関数を適用しているので，外側では適用しない
        h = F.leaky_relu(self.bn2(self.deconv2(h)))
        h = self.rb2(h) # 同上
        h = F.leaky_relu(self.bn3(self.deconv3(h)))
        h = self.rb3(h) # 同上
        h = F.leaky_relu(self.bn4(self.deconv4(h)))
        h = self.rb4(h) # 同上
        y = torch.sigmoid(self.conv(h))
        return y

##### 訓練データセットの読み込み

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from mylib.data_io import CSVBasedDataset


# CSVファイルを読み込み, 訓練データセットを用意
dataset = CSVBasedDataset(
    filename = DATASET_CSV,
    items = [
        'File Path', # X
        TARGET_ATTRIBUTES # Y
    ],
    dtypes = [
        'image', # Xの型
        'float'  # Yの型
    ],
    dirname = DATA_DIR,
    img_transform = transforms.CenterCrop((H, W)) # 処理量を少しでも抑えるため，画像中央の H×W ピクセルの部分だけを対象とする
)

# 訓練データセットを分割し，一方を検証用に回す
dataset_size = len(dataset)
valid_size = int(0.002 * dataset_size) # 全体の 0.2% を検証用に -> tinyCelebA の画像は全部で 16000 枚なので，検証用画像は 16000*0.002=32 枚
train_size = dataset_size - valid_size # 残りの 99.8% を学習用に
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

# 訓練データおよび検証用データをミニバッチに分けて使用するための「データローダ」を用意
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

##### 学習処理の実行

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from mylib.loss_functions import VAELoss
from mylib.visualizers import LossVisualizer
from mylib.data_io import show_images


# ニューラルネットワークの作成
enc_model = FaceEncoder(C=C, H=H, W=W, N=N, K=len(TARGET_ATTRIBUTES)).to(DEVICE)
dec_model = FaceDecoder(C=C, H=H, W=W, N=N, K=len(TARGET_ATTRIBUTES)).to(DEVICE)

# 最適化アルゴリズムの指定（ここでは SGD でなく Adam を使用）
enc_optimizer = optim.Adam(enc_model.parameters())
dec_optimizer = optim.Adam(dec_model.parameters())

# 損失関数
loss_func = VAELoss(channels=3, alpha=0.01)

# 損失関数値を記録する準備
loss_viz = LossVisualizer(['train loss', 'valid loss'])

# 勾配降下法による繰り返し学習
for epoch in range(N_EPOCHS):

    print('Epoch {0}:'.format(epoch + 1))

    # 学習
    enc_model.train()
    dec_model.train()
    sum_loss = 0
    for X, Y in tqdm(train_dataloader):
        for param in enc_model.parameters():
            param.grad = None
        for param in dec_model.parameters():
            param.grad = None
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)
        Z, mu, lnvar = enc_model(X, Y) # 入力画像 X と属性ラベル情報 Y を現在のエンコーダに入力し，特徴ベクトル Z を得る
        X_rec = dec_model(Z, Y) # 特徴ベクトル Z と属性ラベル情報 Y を現在のデコーダに入力し，復元画像 X_rec を得る
        loss = loss_func(X, X_rec, mu, lnvar) # 損失関数の現在値を計算
        loss.backward() # 誤差逆伝播法により，個々のパラメータに関する損失関数の勾配（偏微分）を計算
        enc_optimizer.step() # 勾配に沿ってパラメータの値を更新
        dec_optimizer.step() # 同上
        sum_loss += float(loss) * len(X)
    avg_loss = sum_loss / train_size
    loss_viz.add_value('train loss', avg_loss) # 訓練データに対する損失関数の値を記録
    print('train loss = {0:.6f}'.format(avg_loss))

    # 検証
    enc_model.eval()
    dec_model.eval()
    sum_loss = 0
    with torch.inference_mode():
        for X, Y in tqdm(valid_dataloader):
            X = X.to(DEVICE)
            Y = Y.to(DEVICE)
            Z, mu, lnvar = enc_model(X, Y)
            X_rec = dec_model(mu, Y) 
            loss = loss_func(X, X_rec, mu, lnvar)
            sum_loss += float(loss) * len(X)
    avg_loss = sum_loss / valid_size
    loss_viz.add_value('valid loss', avg_loss) # 検証用データに対する損失関数の値を記録
    print('valid loss = {0:.6f}'.format(avg_loss))
    print('')

    # 学習経過の表示
    if epoch == 0:
        show_images(X.to('cpu').detach(), num=BATCH_SIZE, num_per_row=8, title='original', save_fig=False, save_dir=MODEL_DIR)
    show_images(X_rec.to('cpu').detach(), num=BATCH_SIZE, num_per_row=8, title='epoch {0}'.format(epoch + 1), save_fig=False, save_dir=MODEL_DIR)

# 学習結果のニューラルネットワークモデルをファイルに保存
enc_model = enc_model.to('cpu')
dec_model = dec_model.to('cpu')
torch.save(enc_model.state_dict(), MODEL_FILE_ENC)
torch.save(dec_model.state_dict(), MODEL_FILE_DEC)

# 損失関数の記録をファイルに保存
loss_viz.save(v_file=os.path.join(MODEL_DIR, 'loss_graph.png'), h_file=os.path.join(MODEL_DIR, 'loss_history.csv'))

##### 学習済みニューラルネットワークモデルのロード

In [ ]:
import torch


# ニューラルネットワークモデルとその学習済みパラメータをファイルからロード
dec_model = FaceDecoder(C=C, H=H, W=W, N=N, K=len(TARGET_ATTRIBUTES))
dec_model.load_state_dict(torch.load(MODEL_FILE_DEC))

##### テスト処理（正規分布に従って複数の乱数ベクトルをランダムサンプリングし，それをデコーダに通して画像を生成．属性ラベルは固定値で指定）

In [ ]:
import torch
from mylib.data_io import show_images


dec_model = dec_model.to(DEVICE)
dec_model.eval()

# 生成する画像の枚数
n_gen = 32

# 属性ラベルの指定値
# このサンプルコードでは TARGET_ATTRIBUTES = ['Blond_Hair', 'Brown_Hair', 'Black_Hair', 'Gray_Hair', 'Eyeglasses', 'Male', 'Young'] と設定しているので，
#   'Blond_Hair' = 0, # ブロンド髪ではない
#   'Brown_Hair' = 0, # 茶髪ではない
#   'Black_Hair' = 1, # 黒髪である
#   'Gray_Hair'  = 0, # 白髪ではない
#   'Eyeglasses' = 0, # 眼鏡やサングラスをかけていない
#   'Male'       = 0, # 男性でいない（== 女性）
#   'Young'      = 1, # 若い
# という意味になる
attributes = [1, 0, 0, 0, 1, 0, 1]

# 標準正規分布 N(0, 1) に従って適当に乱数ベクトルを作成
Z = torch.randn((n_gen, N)).to(DEVICE)

# 属性ラベル情報の作成
Y = torch.tensor([attributes], dtype=torch.float32).repeat((n_gen, 1)).to(DEVICE)

# 乱数ベクトルと属性ラベルをデコーダに入力し，その結果を表示
with torch.inference_mode():
    X = dec_model(Z, Y)
    show_images(X.to('cpu').detach(), num=n_gen, num_per_row=8, title='CVAE_sample_generated_case1', save_fig=True)

##### テスト処理（乱数ベクトルを一つだけサンプリングし，それをデコーダに通して画像を生成．属性ラベルは，一つの次元を徐々に変化させる形で指定）

In [ ]:
import copy
import torch
from mylib.data_io import show_images


dec_model = dec_model.to(DEVICE)
dec_model.eval()

# ベースとなる属性ラベル
base_attributes = [0, 0, 1, 0, 0, 0, 1]

# 上の属性ラベルのうち何番目の属性値を変化させるか
# 以下の例は
#   - 0番目の属性（ 'Blond_Hair', ベース値 0 ）を 0 から 1 に徐々に変化
#   - 2番目の属性（ 'Black_Hair', ベース値 1 ）を 1 から 0 に徐々に変化
# という意味になり，すなわち，ブロンド髪から黒髪への属性変化に相当
targets = [5]

# 生成する画像の枚数
n_gen = 16 # 上で指定した属性ラベルを 0～1 の間で n_gen 段階に変化させる

# 標準正規分布 N(0, 1) に従って適当に乱数ベクトルを作成
Z = torch.randn((1, N)).repeat((n_gen, 1)).to(DEVICE)

# 属性ラベル情報の作成
Y = []
for i in range(n_gen):
    attributes = copy.deepcopy(base_attributes)
    for t in targets:
        # t番目の属性値を 0〜1 の範囲でずらす
        if base_attributes[t] == 0:
            attributes[t] = i / (n_gen - 1)
        else:
            attributes[t] = 1 - i / (n_gen - 1)
    Y.append(attributes)
Y = torch.tensor(Y, dtype=torch.float32).to(DEVICE)

# 乱数ベクトルと属性ラベルをデコーダに入力し，その結果を表示
with torch.inference_mode():
    X = dec_model(Z, Y)
    show_images(X.to('cpu').detach(), num=n_gen, num_per_row=8, title='CVAE_sample_generated_case2', save_fig=True)